In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-sample.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-labels.pkl
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-wav.tar

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import malaya_speech
malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.stream` is not able to use.


<module 'malaya_speech' from '/home/husein/dev/malaya-speech/malaya_speech/__init__.py'>

In [4]:
model = malaya_speech.speaker_vector.nemo('huseinzol05/nemo-ecapa-tdnn')

In [5]:
model.eval()

Model(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ECAPAEncoder(
    (layers): ModuleList(
      (0): TDNNModule(
        (conv_layer): Conv1d(80, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
        (activation): ReLU()
        (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): TDNNSEModule(
        (group_tdnn_block): Sequential(
          (0): TDNNModule(
            (conv_layer): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
            (activation): ReLU()
            (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,), groups=8)
          (2): ReLU()
          (3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): TDNNModule(
            (conv_layer): Conv1d(1024, 1024, kernel_size

In [6]:
_ = model.cuda()

In [7]:
import json
import pickle
from tqdm import tqdm

In [8]:
with open('voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [9]:
with open('voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [10]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))

In [11]:
len(unique_files)

36237

In [12]:
import torch
import numpy as np

In [16]:
vectors = {}
for f in tqdm(unique_files):
    y_, _ = malaya_speech.load(f.replace('/home/husein/youtube/', '/home/husein/'))
    try:
        v = model([y_])[0]
        vectors[f] = v
    except:
        torch.cuda.empty_cache()

100%|█████████████████████████████████████| 36237/36237 [08:08<00:00, 74.11it/s]


In [17]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    if labels[i][1] in vectors and labels[i][2] in vectors:
        ls.append(labels[i][0])
        scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|█████████████████████████████| 5900000/5900000 [00:15<00:00, 374078.90it/s]


In [18]:
len(scores), len(vectors)

(5900000, 36237)

In [19]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [20]:
calculate_eer(ls, scores)

(0.024920000000000775, array(0.03515405))

In [21]:
v.shape

(192,)